In [1]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt

In [2]:
df = pd.read_csv('./data/review_data_2t.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141254 entries, 0 to 141253
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   review  141254 non-null  object
 1   시설분류    141254 non-null  object
dtypes: object(2)
memory usage: 2.2+ MB


In [5]:
df.head(2)

,review,시설분류
0,?,문화
1,친절해요,문화


In [6]:
df.columns = ['review','label']

In [7]:
df.head(2)

,review,label
0,?,문화
1,친절해요,문화


In [8]:
# 정규 표현식을 사용하여 한글만 추출
def extract_korean(text):

    # 한글과 공백만 추출
    text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\s]','',text)
    # 공백 2개 이상인경우 1개로 변환
    text = re.sub(r'\s+',' ',text)

    return text

In [27]:
# 형태소 분석 및 특정 품사 추출 함수
def extract_pos(text) :
    okt = Okt()
    pos_data = okt.pos(text)
    adj = ' '.join([word for word, tag in pos_data if tag in ['Adjective']])
    verb = ' '.join([word for word, tag in pos_data if tag in ['Verb']])
    noun = ' '.join([word for word, tag in pos_data if tag in ['Noun']])
    exclamation = ' '.join([word for word, tag in pos_data if tag in ['Exclamation']])
    
    # 각 품사에 대해 값이 없는 경우 NaN 할당
    adj = adj if adj else np.nan
    verb = verb if verb else np.nan
    noun = noun if noun else np.nan
    exclamation = exclamation if exclamation else np.nan
    
    return adj, verb, noun, exclamation                        
                            

In [28]:
# 한글만 추출하여 새로운 컬럼에 저장
df['review2'] = df['review'].map(extract_korean)

In [38]:
df_test = df.head(5).copy()
display(df_test)
df_test[['adj','verb','noun','exclamation']] = df_test['review2'].apply(extract_pos).apply(pd.Series)
df_test

,review,label,review2
0,?,문화,
1,친절해요,문화,친절해요
2,좋아요 만족스러워요,문화,좋아요 만족스러워요
3,와우~~,문화,와우
4,굿,문화,굿


,review,label,review2,adj,verb,noun,exclamation
0,?,문화,,NaN,NaN,NaN,NaN
1,친절해요,문화,친절해요,친절해요,NaN,NaN,NaN
2,좋아요 만족스러워요,문화,좋아요 만족스러워요,좋아요 만족스러워요,NaN,NaN,NaN
3,와우~~,문화,와우,NaN,NaN,와우,NaN
4,굿,문화,굿,NaN,NaN,굿,NaN


In [40]:
from tqdm import tqdm
tqdm.pandas()
# 형용사,동사,명사, 감탄사 추출
df[['adj','verb','noun','exclamation']] = df['review2'].progress_apply(extract_pos).progress_apply(pd.Series)

100%|████████████████████████████████████████████████████████████████████████| 141254/141254 [00:34<00:00, 4115.44it/s]


In [41]:
df.to_csv('./data/review_data_2t_1.csv',index=False)

In [43]:
df2 = pd.read_csv('./data/review_data_2t_1.csv',encoding='utf-8')

In [44]:
df2

,review,label,review2,adj,verb,noun,exclamation
0,?,문화,NaN,NaN,NaN,NaN,NaN
1,친절해요,문화,친절해요,친절해요,NaN,NaN,NaN
2,좋아요 만족스러워요,문화,좋아요 만족스러워요,좋아요 만족스러워요,NaN,NaN,NaN
3,와우~~,문화,와우,NaN,NaN,와우,NaN
4,굿,문화,굿,NaN,NaN,굿,NaN
...,...,...,...,...,...,...,...
141249,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,동물병원,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,꼼꼼히 다행히 좋,해서 갔어요 만져 살펴보시더니 먹어도 되겠다고 하셔서 맞고 왔는데 졌어요,설사 병원 배 보고 약 주사 집 그다음 날,아
141250,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립...,동물병원,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립니다,NaN,데려온 세세하게 봐주셔서 드립니다,저희 천사 곳 중성 혈액 검사 전부 늘 감사,NaN
141251,친절하십니다,동물병원,친절하십니다,친절하십니다,NaN,NaN,NaN
141252,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,동물병원,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,꼼꼼히 약발라주고 있는데,봐주세요 챙겨주시고 가자고하니 따라가네요 해주신 좋아졌어요,진료 진료 후 간식 선생님 산책 요미 처방 습진 약 눈꼽끼는,NaN


In [88]:
# NaN 값제거 후 오름차순 정렬 후 공백을 구분자로 한개의 문자열로 합친후 공백을 구분자로 리스트 변환
adj_list = ' '.join(df2['adj'].dropna().sort_values()).split()
print(len(adj_list))

235178


In [89]:
# 문자열 길이가 1인 요소를 제외하고 새 리스트 생성
filtered_adj_list = [word for word in adj_list if len(word) > 1]

In [92]:
# set로 변환후 중복 제거
deduplicated_adj_list = sorted(list(set(filtered_adj_list)))

In [93]:
deduplicated_adj_list

['가까',
 '가까와',
 '가까우',
 '가까우니',
 '가까우니까',
 '가까우니깐',
 '가까우면',
 '가까우면서',
 '가까우시다면',
 '가까우시면',
 '가까우어요',
 '가까운',
 '가까운거리라',
 '가까운건',
 '가까운게',
 '가까운대',
 '가까운데',
 '가까운데가',
 '가까운데로',
 '가까운데를',
 '가까운데서',
 '가까운데에',
 '가까운으로',
 '가까울수록',
 '가까워',
 '가까워도',
 '가까워서',
 '가까워서도',
 '가까워서요',
 '가까워요',
 '가까웠고',
 '가까웠는데',
 '가까웠습니다',
 '가까웠어요',
 '가까웠었는데',
 '가까웠으면',
 '가까웠음에도',
 '가까으면',
 '가까이있어서',
 '가까히',
 '가까히에서',
 '가깝',
 '가깝게',
 '가깝고',
 '가깝고이',
 '가깝구',
 '가깝구요',
 '가깝기',
 '가깝기만',
 '가깝길래',
 '가깝네요',
 '가깝다',
 '가깝다고',
 '가깝다는',
 '가깝다는거',
 '가깝다면',
 '가깝도',
 '가깝습니다',
 '가깝이',
 '가깝지',
 '가깝지는',
 '가깝지도',
 '가깝지만',
 '가깝지요',
 '가깝진',
 '가난한',
 '가늘게',
 '가늘고',
 '가능하',
 '가능하거나',
 '가능하게',
 '가능하게끔',
 '가능하고',
 '가능하고요',
 '가능하구',
 '가능하구요',
 '가능하기',
 '가능하네요',
 '가능하니',
 '가능하니까',
 '가능하다',
 '가능하다고',
 '가능하다구',
 '가능하다는',
 '가능하다니',
 '가능하다니깐',
 '가능하단다',
 '가능하답니다',
 '가능하더라고',
 '가능하더라구요',
 '가능하도록',
 '가능하며',
 '가능하면',
 '가능하면서',
 '가능하세요',
 '가능하시다고',
 '가능하신',
 '가능하실까',
 '가능하여',
 '가능하여서',
 '가능하죠',
 '가능하지만',
 '가능한',
 '가능한거면',
 '가능한건',
 '가능한것도',
 '가능한게'